## Initialize

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)
library(ggalt)
library(ggdist)
library(ggtext)
library(ggforestplot)
library(cowplot)
library("jsonlite")

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "220126"

In [ ]:
DSM = "MultiTaskSurvivalTraining"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))
#predictions = arrow::read_feather(glue("{data_results_path}/predictions_210803_metabolomics.feather")) 
predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics.feather")) 
loghazards = arrow::read_feather(glue("{data_results_path}/loghazards_model_{run}_metabolomics.feather")) %>% 
    pivot_longer(starts_with("logh"), names_to=c("endpoint", "features"), values_to="logh", names_pattern="logh_?(.*)_(.*)$")

In [ ]:
data_events = data %>% select(eid, ends_with("event"), ends_with("event_time")) %>% 
    pivot_longer(-eid, names_to=c("endpoint", "type"), values_to="value", names_pattern="(.*)(event_time|event)") %>% 
    mutate(endpoint = stringr::str_sub(endpoint, end=-2)) %>% pivot_wider(names_from="type", values_from="value")

In [ ]:
loghazards_tte = loghazards %>% left_join(data_events, by=c("endpoint", "eid"))

In [ ]:
logh_T = loghazards_tte %>% filter(split=="test") %>% mutate(hr=exp(logh))

In [ ]:
logh_T = logh_T %>% group_by(endpoint, features) %>% mutate(logh_perc = ntile(logh, 100))
logh_T_agg = logh_T %>% group_by(endpoint, features) %>% mutate(logh_perc = ntile(logh, 100)) %>% group_by(endpoint, features, logh_perc) %>% summarise(ratio = mean(event))

In [ ]:
labels = logh_T %>% group_by(endpoint, event) %>% summarise(median_logh = mean(logh)) %>% pivot_wider(names_from="event", values_from="median_logh") %>% mutate(delta = `1`-`0`) %>% arrange(desc(delta))

In [ ]:
scores = c("COX_Age+Sex", 
                "COX_Metabolomics",
                "DS_Metabolomics",
                "DS_Age+Sex+Metabolomics",
               "DS_AgeSexMetabolomics")

In [ ]:
colors_path = "colors.json"
colors_dict = read_json(colors_path)

In [ ]:
color_map <- c("all" = "grey", "none" = "black",

               "SCORE_SCORE2" = colors_dict$pastel$grey$light, 
               'SCORE_ASCVD' = colors_dict$pastel$grey$mid, 
                   
               'COX_Age+Sex' = colors_dict$pastel$grey$light, 
               
               'PCA_Metabolomics' = "#4F8EC1",
               'COX_Metabolomics' = "#4F8EC1",
               'DS_Metabolomics' = "#4F8EC1",  #colors_dict$pastel$orange$dark,
                
                'COX_SCORE2' = colors_dict$pastel$grey$light, 
               'COX_ASCVD' = colors_dict$pastel$grey$light, 
               'COX_PANEL' = colors_dict$pastel$grey$light, 

               'DS_Age+Sex+Metabolomics' = "#53dd6c",#colors_dict$pastel$orange$mid,
               'DS_SCORE2+Metabolomics' = colors_dict$pastel$red$mid,
               'DS_ASCVD+Metabolomics' = "#d8315b",#colors_dict$pastel$red$mid,

               'DS_PANEL+Metabolomics' = "#1e1b18" #colors_dict$pastel$red$dark
      )

In [ ]:
scores_full = names(color_map)

In [ ]:
name = glue("benchmark_cindex_{run}")
benchmark_cindex_general = read_feather(glue("{data_results_path}/{name}.feather"))  %>% distinct() %>% unite("score", c(module, features), remove=FALSE) %>%  distinct()

In [ ]:
base_size = 8
title_size = 8
facet_size = 8
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=8), axis.text=element_text(size=8, color="black"), axis.text.x=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2)))

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer"
                   
)

In [ ]:
em_wrap = endpoint_map
em_wrap[names(endpoint_map)] <- str_wrap(unname(endpoint_map), 20)
em_wrap

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
library(ggbeeswarm)
scores_plot = names(color_map)

temp = benchmark_cindex_general %>% 
    mutate_at(vars(score), list(~ factor(., levels=scores_full))) %>% filter(score==score) %>% filter(score %in% scores_plot)
temp_wide = temp %>% select(-module, -features) %>% pivot_wider(names_from="score", values_from="cindex") %>% mutate(delta=`DS_Metabolomics`-`COX_Age+Sex`)
temp_desc = temp %>% group_by(endpoint, score) %>% summarise(mean=median(cindex, na.rm=TRUE), max = max(cindex), .groups="drop")
temp_desc = temp_desc %>% select(-max) %>% pivot_wider(names_from="score", values_from="mean") %>% mutate(delta=`DS_Metabolomics`-`COX_Age+Sex`) %>% mutate(endpoint=fct_reorder(endpoint, desc(delta)))
endpoint_order_desc = levels(temp_desc$endpoint)

In [ ]:
scores_map = c(
    "DS_Metabolomics" = "MET",
    
    "COX_Age+Sex" = "AgeSex",
    "DS_Age+Sex+Metabolomics" = "AgeSex+MET",
    
    "DS_ASCVDnoblood+Metabolomics" = "ASCVD(noblood)+MET",
    
    "COX_ASCVD" = "ASCVD",
    "DS_ASCVD+Metabolomics" = "ASCVD+MET",
    
    "COX_PANEL" = "PANEL", 
    "DS_PANEL+Metabolomics" = "PANEL+MET"
               )

In [ ]:
endpoint_selection = (temp_wide %>% mutate(delta=`DS_Age+Sex+Metabolomics`-`COX_Age+Sex`) %>% 
    group_by(endpoint) %>% median_qi(delta) %>% filter(.lower>0))$endpoint

## Trajectories

In [ ]:
partial_effects = arrow::read_feather(glue("{data_results_path}/adj_partial_effects_metabolomics_{run}.feather"))

In [ ]:
temp_prep = partial_effects %>% #filter(partition==0) %>% 
    filter(quantile %in% c(0.1, 0.5, 0.9)) %>% 
    filter(features %in% c("Age+Sex", "ASCVD", "PANEL")) %>%
    mutate(time=ntile(time, round(max(time)*10, 1))/10) %>% 
    group_by(endpoint, features, quantile, time) #%>% sample_n(27, replace=TRUE)

In [ ]:
purrr::map_dfr(integer(1000), ~ temp_prep %>% sample_n(22, replace=TRUE), .id="i") -> temp_prep_bs
    
temp_prep_bs %>% head()

In [ ]:
trajectory_plots = list()
ncol=3
i=0
endpoint_order_panel = (temp_wide %>% group_by(endpoint) %>% 
                        mutate(delta = `DS_PANEL+Metabolomics`-`COX_PANEL`) %>% 
                        summarise(delta=mean(delta)) %>% arrange(delta))$endpoint
ep_sel = intersect(rev(endpoint_order_panel), endpoint_selection)
for (endpoint in ep_sel){
    temp_time = temp_prep %>% ungroup() %>% 
        filter(endpoint==!!endpoint) %>% #sample_n(1000) %>%# mutate(time=ntile(time, round(max(time)*10, 1))/10) %>%
        group_by(endpoint, features, quantile, time) %>% 
        median_qi(Ft, na.rm=TRUE) # %>% ungroup() %>% sample(1000)
    
trajectory_plots[[endpoint]] = ggplot(temp_time, aes(x = time, y=Ft, color=fct_rev(factor(quantile)), fill=fct_rev(factor(quantile)))) +
        labs(title=endpoint_map[[endpoint]])+
        theme(plot.title = element_text(size=facet_size, hjust=0.5))+
        geom_lineribbon(aes(ymin = .lower, ymax = .upper), size=0.5)+

        labs(x="Time [Years]", y="Cumulative Risk [%]")+

        scale_x_continuous(expand=c(0, 0))+
        scale_y_continuous(labels = function(x) round(x*100, 1), expand=c(0, 0))+
        scale_color_manual(values=c(`0.1`="#7AC6FF", `0.5`="#4F8EC1", `0.9`="#023768"))+ 
        scale_fill_manual(values=c(`0.1`=alpha("#7AC6FF", 0.3), `0.5`=alpha("#4F8EC1", 0.3), `0.9`=alpha("#023768", 0.3)))+                    
                           theme(legend.position="none")+               
        facet_grid(~factor(features))+theme(axis.title.y = element_blank(), axis.title.x = element_blank())

    if (i>=2){trajectory_plots[[endpoint]] = trajectory_plots[[endpoint]] + theme(strip.text.x = element_blank())}
    i=i+1
}

In [ ]:
plot_width=4.25; plot_height=10; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
trajectories = wrap_plots(trajectory_plots, ncol=2)
trajectories

In [ ]:
plot_name = "Figures_4_A_AdjustedTrajectories_long"
trajectories %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device="pdf", width=plot_width, height=plot_height, dpi=plot_dpi)

## Calculate HRs

In [ ]:
temp_agg = temp_diff_calc %>% 
    pivot_longer(-c(endpoint, iteration), names_to="comparison", values_to="delta") %>% group_by(endpoint, comparison) %>% 
    median_qi(delta) %>%
    mutate(different = case_when((0!=delta & 0 >= `.lower` & 0 <= `.upper`) ~ "", TRUE ~ "*")) %>%
    mutate(delta_cindex = glue("{round(delta, 3)} ({round(.lower, 3)}, {round(.upper, 3)}){different}")) %>%
    select(endpoint, comparison, delta_cindex) %>% pivot_wider(names_from="comparison", values_from="delta_cindex") 
temp_agg$endpoint = recode(temp_agg$endpoint, !!!endpoint_map)

In [ ]:
hrs = arrow::read_feather(glue("{data_results_path}/hrs_metabolomics_{run}.feather")) %>% 
    mutate(features = str_remove_all(str_sub(str_remove(str_remove(features, endpoint), "DS__"), end=-3), "_")) %>%
    filter(str_detect(covariate, "logh_"))

In [ ]:
hrs %>% group_by(endpoint, features) %>% median_qi(`exp(coef)`, na.rm=TRUE) %>% mutate(HR_Metabolomics = glue("{`exp(coef)`} ({.lower}, {.upper})")) %>% 
    select(endpoint, features, HR_Metabolomics) 

In [ ]:
ep_sel = c(
    'M_type_2_diabetes',
'M_all_cause_dementia',
'M_heart_failure',
'M_chronic_obstructuve_pulmonary_disease',
'M_renal_disease',
'M_coronary_heart_disease',
'M_fractures',
'M_peripheral_arterial_disease',
'M_liver_disease',
'M_MACE',
'M_asthma',
'M_venous_thrombosis',
'M_cerebral_stroke',
'M_atrial_fibrillation',
'M_cataracts',
'M_prostate_cancer',
'M_abdominal_aortic_aneurysm',
'M_lung_cancer')

In [ ]:
purrr::map_dfr(integer(1000), ~ hrs %>% group_by(endpoint, features) %>% sample_frac(1, replace=TRUE), .id="i") -> temp_hrs

In [ ]:
temp_hrs_agg = temp_hrs %>% filter(endpoint %in% ep_sel) %>%
    group_by(endpoint, features) %>% median_qi(`exp(coef)`, na.rm=TRUE) %>% rename(hr = `exp(coef)`) %>%
    mutate(hr = round(hr, 2), .lower=round(.lower, 2), .upper=round(.upper, 2)) %>%#c(hr, .lower, .upper), round, 1) %>%
    mutate(HR_Metabolomics = glue("{hr} ({.lower}, {.upper})")) %>% 
    select(endpoint, features, HR_Metabolomics) %>%
    pivot_wider(names_from="features", values_from="HR_Metabolomics") %>% 
    select(all_of(c("endpoint", 'Age+Sex+Metabolomics', 
                    #'SCORE2+Metabolomics', 
                    'ASCVD+Metabolomics', 
                   "PANEL+Metabolomics"))) %>% 
    mutate(endpoint = factor(endpoint, levels = endpoint_order)) %>% arrange(endpoint) %>%
    mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>%
    ungroup()
temp_hrs_agg

In [ ]:
fts_map = c(
    "Metabolomics" = "MET",
    "COX_Age+Sex" = "AgeSex",
    "Age+Sex+Metabolomics" = "AgeSex+MET",
    "COX_ASCVD" = "ASCVD",
    "ASCVD+Metabolomics" = "ASCVD+MET",
    "COX_PANEL" = "PANEL", 
    "PANEL+Metabolomics" = "PANEL+MET"
               )

In [ ]:
hr_labels = temp_hrs_agg %>% 
    pivot_longer(-endpoint, names_to="features", values_to="HR")
hr_labels

In [ ]:
temp = temp_hrs %>% filter(endpoint %in% ep_sel)   %>%  mutate(endpoint = factor(endpoint, levels = ep_sel)) %>% arrange(endpoint) %>%
    mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>% filter(features %in% c("Age+Sex+Metabolomics", "ASCVD+Metabolomics", "PANEL+Metabolomics")) %>%
    ungroup() 
temp %>% head()

In [ ]:
## 66%, 95%
#endpoint_order_panel = (temp_wide %>% group_by(endpoint) %>% mutate(delta = `DS_PANEL+Metabolomics`-`COX_PANEL`) %>% summarise(delta=mean(delta)) %>% arrange(delta))$endpoint
plot_width=4; plot_height=10; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)

hr_plot = ggplot(temp, aes(x=fct_rev(factor(features, levels=c("Age+Sex+Metabolomics", "ASCVD+Metabolomics", "PANEL+Metabolomics"))), y=`exp(coef)`))+#levels=endpoint_order_desc))) +
    labs(x=NULL, y="Adjusted Hazard Ratio / SD metabolomic state")+
    #geom_boxplot()+
    stat_gradientinterval(color="black", alpha=0.7, fatten_point=0.8, interval_size_range=c(0.3, 0.6), fill="white")+
    geom_hline(yintercept=1, alpha=0.7, color="black", size=0.25)+
    geom_hline(yintercept=1.5, alpha=0.3, color="black", size=0.25, linetype="22")+
    geom_hline(yintercept=2, alpha=0.4, color="black", size=0.25, linetype="22")+
    geom_hline(yintercept=3, alpha=0.5, color="black", size=0.25, linetype="22")+
    scale_x_discrete(labels=fts_map)+
    coord_flip(ylim=c(-0.6, 4))+#ylim=c(-0.02, 0.02))+
    #scale_y_continuous()+ 
    theme(plot.title = element_text(hjust=0.5))+
    facet_grid(rows=vars(endpoint), space="free", switch = "y") +
      theme(strip.placement = "outside")+
    theme(strip.text.y.left = element_text(angle = 0))+
    theme(axis.text.y = element_text(size=5.5))+
    geom_text(data=hr_labels, mapping=aes(x=fct_rev(factor(features, levels=c("Age+Sex+Metabolomics", "ASCVD+Metabolomics", "PANEL+Metabolomics"))), label=glue("HR: {HR}")), y=-0.65, size=2, hjust=0)
hr_plot

In [ ]:
plot_name = "Figures_4_B_AdjustedHRs_long"
hr_plot %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device="pdf", width=plot_width, height=plot_height, dpi=plot_dpi)